In [1]:
from openai import OpenAI
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_random_exponential
import time
import random

In [2]:
#api key is hidden for safety
api_key = 'sk-proj-XMyCCNFZ5QccTMeLDt4kT3BlbkFJe5QTcV0hvrbCMTptYM8C'
client = OpenAI(api_key = api_key)

In [3]:
data = pd.read_pickle('data_cleaned.pkl')

In [35]:
#function to call the api with Open AI, provide a review and receive a sentiment (positive or negative) in return.
def get_sentiment(review):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Please classify the sentiment of this movie review as positive or negative and return only the word Positive or Negative: \"{review}\""}
        ],
        model="gpt-3.5-turbo"
    )
    
    try:
        sentiment = response.choices[0].message.content.strip().lower()
        return sentiment
    except AttributeError as e:
        print(f"AttributeError: {e}")
        return None

In [36]:
#testing with a single review 
test_review = 'good movie'
sentiment = get_sentiment(test_review)

In [37]:
print(sentiment)

positive


In [38]:
data_subset = data.head(10)
data_subset

,review,sentiment,tokens
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, 1, oz, e..."
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, filming, techn..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,negative,"[basically, theres, family, little, boy, jake,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, matteis, love, time, money, visually,..."
5,"Probably my all-time favorite movie, a story o...",positive,"[probably, alltime, favorite, movie, story, se..."
6,I sure would like to see a resurrection of a u...,positive,"[sure, would, like, see, resurrection, dated, ..."
7,"This show was an amazing, fresh & innovative i...",negative,"[show, amazing, fresh, innovative, idea, 70s, ..."
8,Encouraged by the positive comments about this...,negative,"[encouraged, positive, comments, film, looking..."
9,If you like original gut wrenching laughter yo...,positive,"[like, original, gut, wrenching, laughter, lik..."


In [42]:
#testing out my functions with a simple 10 sample subset of my data
def analyze_reviews(data_subset):
    data_subset['ai_response'] = data_subset['review'].apply(lambda x: get_sentiment(x))
    return data_subset

In [43]:
result = analyze_reviews(data_subset)
print(result)

                                              review sentiment  \
0  One of the other reviewers has mentioned that ...  positive   
1  A wonderful little production. <br /><br />The...  positive   
2  I thought this was a wonderful way to spend ti...  positive   
3  Basically there's a family where a little boy ...  negative   
4  Petter Mattei's "Love in the Time of Money" is...  positive   
5  Probably my all-time favorite movie, a story o...  positive   
6  I sure would like to see a resurrection of a u...  positive   
7  This show was an amazing, fresh & innovative i...  negative   
8  Encouraged by the positive comments about this...  negative   
9  If you like original gut wrenching laughter yo...  positive   

                                              tokens ai_response  
0  [one, reviewers, mentioned, watching, 1, oz, e...    negative  
1  [wonderful, little, production, filming, techn...    positive  
2  [thought, wonderful, way, spend, time, hot, su...    positive  
3  [b

C:\Users\Andrew\AppData\Local\Temp\ipykernel_24968\3534760871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset['ai_response'] = data_subset['review'].apply(lambda x: get_sentiment(x))


In [44]:
result[['sentiment', 'ai_response']]

,sentiment,ai_response
0,positive,negative
1,positive,positive
2,positive,positive
3,negative,negative
4,positive,positive
5,positive,positive
6,positive,positive
7,negative,negative
8,negative,negative
9,positive,positive


In [46]:
#The model was timing out when attempting to do the entire 50,000 row dataset, so I took a 1000 sample subset to test the model
data_subset = data.head(1000)
result = analyze_reviews(data_subset)
result[['sentiment', 'ai_response']]

C:\Users\Andrew\AppData\Local\Temp\ipykernel_24968\3534760871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset['ai_response'] = data_subset['review'].apply(lambda x: get_sentiment(x))


,sentiment,ai_response
0,positive,negative
1,positive,positive
2,positive,positive
3,negative,negative
4,positive,positive
...,...,...
995,positive,positive
996,negative,negative
997,negative,negative
998,negative,negative


In [50]:
#determining the accuracy 
count_not_equal = (result['sentiment'] != result['ai_response']).sum()
accuracy = 1 - count_not_equal/len(result['sentiment'])
print(accuracy)

0.9410000000000001


In [ ]:
#The AI model does prove to be a better model in determining the sentiment of a movie review than the models that I built myself
# by an impressive 9%.  This method boasts an accuracy of 94%.